In [20]:
import pandas as pd
import json
import numpy as np
import jenks

In [21]:
momc = pd.read_csv('../scratch/momc_meta.csv').set_index('KOD_MOaMC').to_dict(orient='index')
ob = pd.read_csv('../scratch/obce_meta.csv').set_index('KOD_OBEC').to_dict(orient='index')

In [22]:
with open('../scratch/okrsky.json', encoding='utf-8') as f:
    jsn = json.load(f)

In [23]:
meta = {}

In [24]:
# zpracovani MC
for mc in momc:
    meta[mc] = {
        'mc': momc[mc]['NAZ_MOaMC'], 
        'obec': momc[mc]['NAZ_OBEC'], 
        'okres': momc[mc]['NAZ_LAU1'], 
        'kraj': momc[mc]['NAZ_CZNUTS3']
    }

In [25]:
# zpracovani obci
for o in ob:
    meta[o] = {
        'mc': None, 
        'obec': ob[o]['NAZ_OBEC'], 
        'okres': ob[o]['NAZ_LAU1'], 
        'kraj': ob[o]['NAZ_CZNUTS3']
    }

In [26]:
for fe in jsn['features']:
    momc_kod = fe['properties']['MomcKod']
    obec_kod = fe['properties']['ObecKod']
    if momc_kod != None:
        kd = momc_kod
    else:
        kd = obec_kod
    
    fe['properties'].update(meta[kd])

In [27]:
okrs = pd.read_csv('../scratch/okrsky_temp.csv')

In [28]:
okrs

,OBEC,OKRSEK,DATUM_CAS_ZPRAC,HLASY_02,HLASY_03,HLASY_05,HLASY_07,HLASY_08,HLASY_09,ODEVZ_OBAL,OPAKOVANE,PL_HL_CELK,PORADI_ZPRAC,VYD_OBALKY,VOL_SEZNAM,HLASY_01,HLASY_04,HLASY_06
0,566080,1,2018-01-13T14:21:31,4,1,1,14,2,4,26,0,26,2,26,68,0,0,0
1,573060,5,2018-01-13T14:23:15,7,7,1,32,9,9,69,0,69,3,69,124,2,1,1
2,547701,1,2018-01-13T14:23:18,3,4,1,12,5,15,47,0,47,4,47,66,6,1,0
3,542059,1,2018-01-13T14:23:20,13,1,0,9,0,6,29,0,29,5,29,48,0,0,0
4,599034,1,2018-01-13T14:23:30,7,4,0,18,9,7,47,0,47,6,47,73,2,0,0
5,588261,1,2018-01-13T14:24:01,5,9,0,9,13,18,57,0,57,7,57,75,3,0,0
6,531391,1,2018-01-13T14:24:04,1,0,0,20,3,15,42,0,41,8,42,63,1,0,1
7,999997,52,2018-01-13T14:24:08,4,4,0,3,4,19,37,0,37,9,37,69,2,1,0
8,572241,11,2018-01-13T14:24:09,1,3,0,20,3,9,36,0,36,10,36,45,0,0,0
9,537322,1,2018-01-13T14:24:11,1,2,0,8,0,7,21,0,21,11,21,35,3,0,0


In [29]:
okrs['okid'] = okrs.OBEC.astype('str') + '_' + okrs.OKRSEK.astype('str')

In [31]:
okrs.columns

Index(['OBEC', 'OKRSEK', 'DATUM_CAS_ZPRAC', 'HLASY_02', 'HLASY_03', 'HLASY_05',
       'HLASY_07', 'HLASY_08', 'HLASY_09', 'ODEVZ_OBAL', 'OPAKOVANE',
       'PL_HL_CELK', 'PORADI_ZPRAC', 'VYD_OBALKY', 'VOL_SEZNAM', 'HLASY_01',
       'HLASY_04', 'HLASY_06', 'okid'],
      dtype='object')

In [30]:
cls = [
    'CHYBA',
    'KC_1',
    'KC_2',
    'KC_3',
    'KC_4',
    'HLASY_10',
    'HLASY_11',
    'HLASY_12',
    'HLASY_13',
    'HLASY_14',
    'HLASY_15',
    'HLASY_16',
    'HLASY_17',
    'HLASY_18',
    'HLASY_19',
    'HLASY_20',
    'KC_SUM',
    'KOLO',
    'OBEC',
    'OKRES',
    'OKRSEK',
    'OPRAVA',
    'POSL_KAND',
    'TYP_FORM'
]
okrs.drop(cls, inplace=True, axis=1)

ValueError: labels ['CHYBA' 'KC_1' 'KC_2' 'KC_3' 'KC_4' 'HLASY_10' 'HLASY_11' 'HLASY_12'
 'HLASY_13' 'HLASY_14' 'HLASY_15' 'HLASY_16' 'HLASY_17' 'HLASY_18'
 'HLASY_19' 'HLASY_20' 'KC_SUM' 'KOLO' 'OKRES' 'OPRAVA' 'POSL_KAND'
 'TYP_FORM'] not contained in axis

In [32]:
okrs = okrs.set_index('okid').to_dict(orient='index')

In [33]:
for fe in jsn['features']:
    momc_kod = fe['properties']['MomcKod']
    obec_kod = fe['properties']['ObecKod']
    if momc_kod != None:
        kd = str(momc_kod) + '_' + str(fe['properties']['Cislo'])
    else:
        kd = str(obec_kod) + '_' + str(fe['properties']['Cislo'])
    
    if kd in okrs:
        fe['properties'].update(okrs[kd])
    else:
        print(kd)

579173_1
592935_1
547051_38006
579025_8
579025_12
545422_1
579742_1
579513_1
555177_1
579050_1
503941_1
579092_1
554367_10001
500208_8089


In [34]:
with open('../scratch/map.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(jsn))

In [36]:
#breaks
k = 5
nbAttempt = 4

target = open('../js/breaks.js', 'w')
target.truncate()
target.write('var breaks = {\n')

for i in range(1, 10):
    print('kandidat ' + str(i))
    vals = []
    for ftr in jsn['features']:
        try:
            vals.append(ftr['properties']['HLASY_' + str(i).rjust(2, '0')] / ftr['properties']['PL_HL_CELK'])
        except:
            print('omit')
        
    vals = np.trim_zeros(vals)
    if np.mean(vals) == 0:
        continue
    else:
        brks = [0, 0, 0, 0]
        try:
            casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
            brks = casp.breaks
        except:
            pass
        target.write('"HLASY_' + str(i).rjust(2, '0') + '": ' + str(brks) + ',\n')
        
vals = []
print('ucast')
for ftr in jsn['features']:
    try:
        vals.append(ftr['properties']['PL_HL_CELK'] / ftr['properties']['VOL_SEZNAM'])
    except:
        print('omit')

vals = np.trim_zeros(vals)
if np.mean(vals) == 0:
    pass
else:
    brks = [0, 0, 0, 0]
    try:
        casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
        brks = casp.breaks
    except:
        pass
    target.write('"UCAST": ' + str(brks) + ',\n')       
target.write("};")
target.close()

kandidat 1
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
Running Jenks-Caspall natural breaks...
**Attempt number 1
Step 1 : kmeans++ initalization, GVF = 0.827233
Step 2 : kmeans complete in 51 iterations and 11357 moves, GVF = 0.899482
Step 3 : Forcing completed in 2 cycles and 4 moves, GVF = 0.899482
Running Jenks-Caspall natural breaks...
**Attempt number 2
Step 1 : kmeans++ initalization, GVF = 0.892175
Step 2 : kmeans complete in 24 iterations and 2172 moves, GVF = 0.899522
Step 3 : Forcing completed in 1 cycles and 0 moves, GVF = 0.899522
Running Jenks-Caspall natural breaks...
**Attempt number 3
Step 1 : kmeans++ initalization, GVF = 0.879736
Step 2 : kmeans complete in 42 iterations and 5052 moves, GVF = 0.899508
Step 3 : Forcing completed in 2 cycles and 6 moves, GVF = 0.899508
Running Jenks-Caspall natural breaks...
**Attempt number 4
Step 1 : kmeans++ initalization, GVF = 0.857336
Step 2 : kmeans complete in 50 iterations and 7769 moves, GVF = 0

Step 3 : Forcing completed in 2 cycles and 12 moves, GVF = 0.923730
Running Jenks-Caspall natural breaks...
**Attempt number 2
Step 1 : kmeans++ initalization, GVF = 0.915751
Step 2 : kmeans complete in 24 iterations and 2567 moves, GVF = 0.923714
Step 3 : Forcing completed in 4 cycles and 12 moves, GVF = 0.923714
Running Jenks-Caspall natural breaks...
**Attempt number 3
Step 1 : kmeans++ initalization, GVF = 0.847340
Step 2 : kmeans complete in 47 iterations and 14739 moves, GVF = 0.923714
Step 3 : Forcing completed in 4 cycles and 12 moves, GVF = 0.923714
Running Jenks-Caspall natural breaks...
**Attempt number 4
Step 1 : kmeans++ initalization, GVF = 0.899686
Step 2 : kmeans complete in 42 iterations and 4735 moves, GVF = 0.923730
Step 3 : Forcing completed in 2 cycles and 12 moves, GVF = 0.923730
Jenks-Caspall competed!
14740 values, 5 classes
Breaks with bounds [0.057459677419354836, 0.2703252032520325, 0.3734015345268542, 0.4631578947368421, 0.5681818181818182, 0.891304347826086